In [ ]:
# ai서버 요청 코드

In [2]:
import requests
from aws_requests_auth.aws_auth import AWSRequestsAuth
import json

# AWS 설정 (파일에서 읽기)
with open('../../../../keys/aws_accesskey.txt', 'r') as f:
    access_key = f.read().strip()

with open('../../../../keys/aws_secretkey.txt', 'r') as f:
    secret_key = f.read().strip()

# 기타 AWS 정보
region = 'ap-northeast-2'
service = 'sagemaker'
endpoint = 'runtime.sagemaker.ap-northeast-2.amazonaws.com'
url = 'https://runtime.sagemaker.ap-northeast-2.amazonaws.com/endpoints/dawoncecdEP/invocations'

# AWS Signature V4 인증 설정
auth = AWSRequestsAuth(
    aws_access_key=access_key,
    aws_secret_access_key=secret_key,
    aws_host=endpoint,
    aws_region=region,
    aws_service=service
)

# 요청 본문 (JSON)
data = {
    "text": "sample"
}

# POST 요청 보내기
response = requests.post(url, auth=auth, json=data)

# 응답을 UTF-8로 디코딩하고 JSON으로 파싱
response_text = response.content.decode('utf-8')
response_data = json.loads(response_text)

# 응답 출력
print("Response Code:", response.status_code)
print(json.dumps(response_data, ensure_ascii=False, indent=4))


Response Code: 200
{
    "passage0": "전기 및 낙뢰 손상",
    "sim0": 111.8690185546875,
    "passage1": "익수(침수와 익사)",
    "sim1": 111.62054443359375,
    "passage2": "토혈-혈변",
    "sim2": 111.6041259765625,
    "passage3": "중독",
    "sim3": 111.54843139648438,
    "passage4": "심정지 케이스",
    "sim4": 111.48954772949219,
    "passage5": "알레르기 및 아나필락시스",
    "sim5": 111.43937683105469,
    "script0": {
        "임상적 특징": "전기 화상 전기가 들어가는 곳과 나오는 곳의 심한 조직 손상, 고전압인 경우 심한 근육수축 유발로 심부조직의 파괴와 골절 등 발생, 전기흐름이 심장을 통과할 경우 치명적인 부정맥 발생 낙뢰 손상 천만 볼트 이상의 직류전류가 0.1초 이하의 시간 동안 작용 사망원인은 주로 호흡정지 또는 무수축(asystole) 직접 낙뢰에 맞지 않아도 낙뢰 손상 발생 가능 피부에 특징적인 고사리모양 무늬(Lichtenberg figure)",
        "환자평가 필수항목": "병력청취 전류의 특성: 전압 및 교류/직류 여부, 신체 통과 경로(entry/exit wounds) 및 노출 시간, 손상기전 및 사고발생 당시 상황을 확인한다. 환자의 과거력, 수술력, 약물복용력, 임신여부 등을 확인한다. 이학적 검진 심장 리듬의 평가 및 심전도 감시 추락 등 동반 손상의 확인"
    },
    "script1": {
        "임상적 특징": "물에 빠지게 되면, 숨을 최대한 참다가 물을 폐로 흡입하게 된다. 다량의 물이 흡입되면서 폐부종이 발생하게 되고, 결국은 저산소증이 발생하여 무호흡 및 심정지까지 이어질 수 있다. 건성(dry) 익사는 소

In [ ]:
# 테스트 데이터 불러오는 코드

In [3]:
def load_manual_data(file_paths):
    """
    주어진 파일 목록에서 <docsX/> 태그로 구분된 manual 데이터를 로드하고,
    각각을 리스트의 원소로 처리하는 함수.
    """
    manuals = []
    
    # 각 파일에서 데이터를 읽어와 <docsX/> 태그로 구분된 매뉴얼을 하나씩 처리
    for file_path in file_paths:
        current_manual = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                # <docsX/> 태그를 만나면 이전 매뉴얼을 저장하고 새로운 매뉴얼 시작
                if line.startswith("<docs"):
                    if current_manual:
                        # 이전 매뉴얼을 리스트에 추가
                        manuals.append('\n'.join(current_manual))
                        current_manual = []
                # 내용이 있는 줄은 현재 매뉴얼에 추가
                elif line:
                    current_manual.append(line)

        # 마지막 매뉴얼도 추가
        if current_manual:
            manuals.append('\n'.join(current_manual))

    # 샘플로 첫 3개의 manual을 출력
    for i in range(min(3, len(manuals))):
        print(f"Sample Manual {i+1}:")
        print(manuals[i][:200])  # 첫 200자만 출력
        print("=" * 50)

    return manuals

# manual1.txt와 manual2.txt에서 manual 데이터를 로드하고 통합
manual_files = ['../test_data/manual1.txt', '../test_data/manual2.txt']
manuals = load_manual_data(manual_files)


Sample Manual 1:
심정지 케이스 임상적 특징
질병으로 인하여 24시간 이전에 사망할 것이 예견되지 않던 환자에서 증상발생 2시간 이내에 심정지가 발생한 것을 ‘급성 심정지’ 라고 하여 심인성 심정지일 확률이 높으며 신속한 제세동 치료가 소생술 성공에 결정적이다.
가역적인 원인인 급성심근경색, 고칼륨혈증, 약물중독, 저체온증 등으로 2차적으로 심정지가 발생할 수 있으므로 이러
Sample Manual 2:
비외상성 쇼크 임상적 특징
의식변화: 쇼크로 인해 가장 먼저 뇌가 영향을 받아 안절부절, 불안감을 유발할 수 있고 이는 혼돈, 투쟁으로 발전하며 더욱 심해질 경우무의식 상태가 된다.
피부: 혈관이 수축하면서 피부가 창백하고 차가워지며, 더욱 진행할 경우 땀이 나게 된다.
호흡: 빠르고 얕은 호흡을 하고, 더욱 진행할 경우 호흡수는 감소한다.
맥박: 빠르고 
Sample Manual 3:
이물질에 의한 기도 폐쇄 임상적 특징
부분기도폐쇄의 경우 환자가 기침을 하거나 소리를 낼 수 있으며 협착음 혹은 천명음이 청진된다.
완전기도폐쇄의 경우 양손 혹은 한손으로 목 부위를 감싸 쥐는 ‘질식 징후’ 를 보일 수 있으며 기침을 하거나 목소리를 낼 수 없다.


In [ ]:
manuals[0]

In [4]:
def load_pair_data(file_path):
    """
    m과 d 쌍의 데이터를 불러오는 함수.
    매뉴얼 내용과 대화 내용을 간단한 구조로 저장하고,
    대화는 상황실과 신고자의 발화로 구분하여 리스트로 저장.
    """
    pairs = []
    current_manual_text = []
    current_dialogue_turns = []
    is_manual = True  # 매뉴얼과 대화를 구분하기 위한 플래그

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # 매뉴얼 텍스트가 시작되면 is_manual 플래그를 True로 설정
            if line.startswith("<m"):
                if current_manual_text and current_dialogue_turns:  # 이전 매뉴얼 저장
                    pairs.append({
                        'manual': '\n'.join(current_manual_text),
                        'dialogue': current_dialogue_turns
                    })
                current_manual_text = []
                current_dialogue_turns = []
                is_manual = True
            # 대화 텍스트가 시작되면 is_manual 플래그를 False로 설정
            elif line.startswith("<d"):
                is_manual = False
            elif is_manual:
                # 매뉴얼 내용 추가
                current_manual_text.append(line)
            elif not is_manual and line:
                # 대화를 턴 별로 구분하여 저장
                if "상황실:" in line:
                    #current_dialogue_turns.append(line.replace("상황실:", "").strip())
                    current_dialogue_turns.append(line.strip())
                elif "신고자:" in line:
                    #current_dialogue_turns.append(line.replace("신고자:", "").strip())
                    current_dialogue_turns.append(line.strip())

    # 마지막 매뉴얼과 대화 저장
    if current_manual_text and current_dialogue_turns:
        pairs.append({
            'manual': '\n'.join(current_manual_text),
            'dialogue': current_dialogue_turns
        })

    # 샘플로 첫 3개의 m-d 쌍 출력
    for i, pair in enumerate(pairs[:3]):
        print(f"Sample Pair {i+1}:")
        print("Manual:")
        print(pair['manual'][:200])  # 매뉴얼 텍스트의 첫 200자 출력
        print("Dialogue:")
        print(pair['dialogue'][:6])  # 첫 6턴의 대화 출력
        print("=" * 50)

    return pairs

# manual_question_integrated.txt에서 m과 d 쌍 데이터를 로드
pairs = load_pair_data('../test_data/manual_question_integrated.txt')


Sample Pair 1:
Manual:
심정지 케이스 임상적 특징
질병으로 인하여 24시간 이전에 사망할 것이 예견되지 않던 환자에서 증상발생 2시간 이내에 심정지가 발생한 것을 ‘급성 심정지’ 라고 하여 심인성 심정지일 확률이 높으며 신속한 제세동 치료가 소생술 성공에 결정적이다.
가역적인 원인인 급성심근경색, 고칼륨혈증, 약물중독, 저체온증 등으로 2차적으로 심정지가 발생할 수 있으므로 이러
Dialogue:
['상황실: 네, 119응급센터입니다. 어떤 상황에서 도움이 필요하신가요?', '신고자: 안녕하세요, 여기 공원에서 산책 중인데, 한 사람이 갑자기 쓰러져서 의식이 없습니다. 숨도 쉬지 않는 것 같아요.', '상황실: 현재 위치는 어디신가요?', '신고자: 서울 서대문구 연세로 근처 공원입니다.', '상황실: 신고자님 성함과 연락처를 알려주시겠어요?', '신고자: 제 이름은 이정훈이고, 전화번호는 010-XXXX-XXXX입니다.']
Sample Pair 2:
Manual:
비외상성 쇼크 임상적 특징
의식변화: 쇼크로 인해 가장 먼저 뇌가 영향을 받아 안절부절, 불안감을 유발할 수 있고 이는 혼돈, 투쟁으로 발전하며 더욱 심해질 경우무의식 상태가 된다.
피부: 혈관이 수축하면서 피부가 창백하고 차가워지며, 더욱 진행할 경우 땀이 나게 된다.
호흡: 빠르고 얕은 호흡을 하고, 더욱 진행할 경우 호흡수는 감소한다.
맥박: 빠르고 
Dialogue:
['상황실: 네, 119응급센터입니다. 어떤 상황에서 도움이 필요하신가요?', '신고자: 안녕하세요, 여기 공원에서 산책 중인데, 한 사람이 갑자기 매우 불안해 보이고 의식이 혼미한 상태입니다. 가슴을 움켜쥐고 숨을 잘 쉬지 못하고 있어요.', '상황실: 현재 위치는 어디신가요?', '신고자: 서울 송파구 올림픽공원 근처입니다.', '상황실: 신고자님 성함과 연락처를 알려주시겠어요?', '신고자: 제 이름은 이지수이고, 전화번호는 010-XXXX-XXXX입니다.']
Sample Pair 3:
Manual:
이물

In [5]:
pairs

[{'manual': '심정지 케이스 임상적 특징\n질병으로 인하여 24시간 이전에 사망할 것이 예견되지 않던 환자에서 증상발생 2시간 이내에 심정지가 발생한 것을 ‘급성 심정지’ 라고 하여 심인성 심정지일 확률이 높으며 신속한 제세동 치료가 소생술 성공에 결정적이다.\n가역적인 원인인 급성심근경색, 고칼륨혈증, 약물중독, 저체온증 등으로 2차적으로 심정지가 발생할 수 있으므로 이러한 원인을 찾으려는 노력이 중요하다.',
  'dialogue': ['상황실: 네, 119응급센터입니다. 어떤 상황에서 도움이 필요하신가요?',
   '신고자: 안녕하세요, 여기 공원에서 산책 중인데, 한 사람이 갑자기 쓰러져서 의식이 없습니다. 숨도 쉬지 않는 것 같아요.',
   '상황실: 현재 위치는 어디신가요?',
   '신고자: 서울 서대문구 연세로 근처 공원입니다.',
   '상황실: 신고자님 성함과 연락처를 알려주시겠어요?',
   '신고자: 제 이름은 이정훈이고, 전화번호는 010-XXXX-XXXX입니다.',
   '상황실: 감사합니다. 환자분이 쓰러진 지 얼마나 됐는지 아시나요?',
   '신고자: 쓰러진 지 약 1분 정도 된 것 같습니다.',
   '상황실: 환자분이 현재 의식이 없고, 호흡도 없는 상태라고 하셨는데, 심정지 상태일 가능성이 높습니다. 지금 바로 심폐소생술을 시행할 수 있나요?',
   '신고자: 제가 심폐소생술을 해본 적은 없지만, 해보겠습니다.',
   '상황실: 네, 저와 함께 따라해 주세요. 우선 환자의 가슴 한가운데에 손을 겹쳐 놓고, 팔꿈치를 곧게 펴고 가슴을 강하게 눌러주세요. 분당 100-120회의 속도로 압박해주세요.']},
 {'manual': '비외상성 쇼크 임상적 특징\n의식변화: 쇼크로 인해 가장 먼저 뇌가 영향을 받아 안절부절, 불안감을 유발할 수 있고 이는 혼돈, 투쟁으로 발전하며 더욱 심해질 경우무의식 상태가 된다.\n피부: 혈관이 수축하면서 피부가 창백하고 차가워지며, 더욱 진행할 경우 땀이 나게 된다.\n호흡: 빠르고

In [6]:
def extend_pairs(pairs, start_percentage=0.4):
    """
    pairs에서 각 dialogue를 50% 지점까지 고정한 후, 두 문장씩 늘려가며 데이터를 확장하는 함수.
    """
    pairs_extended = []

    for pair in pairs:
        manual = pair['manual']
        dialogue = pair['dialogue']

        num_sentences = len(dialogue)
        start_index = int(num_sentences * start_percentage)  # 50% 지점

        # 50%까지의 고정된 부분 추가
        fixed_dialogue = dialogue[:start_index]
        pairs_extended.append({
            'manual': manual,
            'dialogue': fixed_dialogue  # 50% 고정 부분
        })

        # 50% 이후로 두 문장씩 확장
        for i in range(start_index, num_sentences, 2):
            extended_dialogue = dialogue[:i + 2]
            pairs_extended.append({
                'manual': manual,
                'dialogue': extended_dialogue
            })

    # 샘플로 첫 3개의 확장된 데이터를 출력
    for i, pair in enumerate(pairs_extended[:3]):
        print(f"Extended Pair {i+1}:")
        print("Manual:")
        print(pair['manual'][:200])  # 첫 200자만 출력
        print("Dialogue:")
        print(pair['dialogue'])  # 대화 문장 전체 출력
        print("=" * 50)

    return pairs_extended

# pairs 확장
pairs_extended = extend_pairs(pairs)


Extended Pair 1:
Manual:
심정지 케이스 임상적 특징
질병으로 인하여 24시간 이전에 사망할 것이 예견되지 않던 환자에서 증상발생 2시간 이내에 심정지가 발생한 것을 ‘급성 심정지’ 라고 하여 심인성 심정지일 확률이 높으며 신속한 제세동 치료가 소생술 성공에 결정적이다.
가역적인 원인인 급성심근경색, 고칼륨혈증, 약물중독, 저체온증 등으로 2차적으로 심정지가 발생할 수 있으므로 이러
Dialogue:
['상황실: 네, 119응급센터입니다. 어떤 상황에서 도움이 필요하신가요?', '신고자: 안녕하세요, 여기 공원에서 산책 중인데, 한 사람이 갑자기 쓰러져서 의식이 없습니다. 숨도 쉬지 않는 것 같아요.', '상황실: 현재 위치는 어디신가요?', '신고자: 서울 서대문구 연세로 근처 공원입니다.']
Extended Pair 2:
Manual:
심정지 케이스 임상적 특징
질병으로 인하여 24시간 이전에 사망할 것이 예견되지 않던 환자에서 증상발생 2시간 이내에 심정지가 발생한 것을 ‘급성 심정지’ 라고 하여 심인성 심정지일 확률이 높으며 신속한 제세동 치료가 소생술 성공에 결정적이다.
가역적인 원인인 급성심근경색, 고칼륨혈증, 약물중독, 저체온증 등으로 2차적으로 심정지가 발생할 수 있으므로 이러
Dialogue:
['상황실: 네, 119응급센터입니다. 어떤 상황에서 도움이 필요하신가요?', '신고자: 안녕하세요, 여기 공원에서 산책 중인데, 한 사람이 갑자기 쓰러져서 의식이 없습니다. 숨도 쉬지 않는 것 같아요.', '상황실: 현재 위치는 어디신가요?', '신고자: 서울 서대문구 연세로 근처 공원입니다.', '상황실: 신고자님 성함과 연락처를 알려주시겠어요?', '신고자: 제 이름은 이정훈이고, 전화번호는 010-XXXX-XXXX입니다.']
Extended Pair 3:
Manual:
심정지 케이스 임상적 특징
질병으로 인하여 24시간 이전에 사망할 것이 예견되지 않던 환자에서 증상발생 2시간 이내에 심정지가 발생한 것을 ‘급성 심정지’ 라고 하

In [7]:
len(pairs_extended)

127

In [ ]:
# 테스팅 코드 시작

In [8]:
# SageMaker로 dialogue 데이터를 테스트하고 결과를 JSON 파일에 저장하는 함수
def test_pairs_extended_and_save_json_sagemaker(pairs_extended, top_k=6, output_file="sim_result_sagemaker.json"):
    results = []
    
    for i, pair in enumerate(pairs_extended):
        # dialogue 리스트를 \n으로 합쳐서 하나의 텍스트로 변환 (query로 사용)
        dialogue_text = "\n".join(pair['dialogue'])
        
        # 요청 본문 (JSON)
        data = {
            "text": dialogue_text
        }
        
        # SageMaker로 POST 요청 보내기
        response = requests.post(url, auth=auth, json=data)
        response_text = response.content.decode('utf-8')
        response_data = json.loads(response_text)
        
        # 각 쌍에 대한 결과를 딕셔너리로 정리
        result = {
            "extended_pair": i + 1,
            "dialogue": pair['dialogue'],
            "top_k_manuals": []
        }
        
        # SageMaker 응답에서 passage0부터 passage5까지 추출하여 저장
        for k in range(top_k):
            passage_key = f"passage{k}"
            similarity_key = f"sim{k}"
            if passage_key in response_data and similarity_key in response_data:
                result['top_k_manuals'].append({
                    "manual_index": k + 1,
                    "manual_text": response_data[passage_key],
                    "similarity": float(response_data[similarity_key])
                })
        
        # 결과 리스트에 추가
        results.append(result)
    
    # JSON 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    
    print(f"Results saved to {output_file}")

# pairs_extended에 대해 테스트 실행하고 결과를 JSON 파일에 저장
test_pairs_extended_and_save_json_sagemaker(pairs_extended, top_k=6, output_file="sim_result_sagemaker.json")

Results saved to sim_result_sagemaker.json
